In [1]:
import pandas as pd 
import numpy as np  
import scipy as sp  
import seaborn as sns
sns.set_style('darkgrid') 

In [2]:
import pickle       
import regex as re  

In [3]:
import gensim
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

In [4]:
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler 
from sklearn.naive_bayes import MultinomialNB  
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
%matplotlib inline

C:\Users\Joginder Bairwa\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler 
from sklearn.naive_bayes import MultinomialNB  
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
%matplotlib inline

In [6]:
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [7]:
# Loads in the data into a Pandas data frame
fcc_csv = pd.read_csv('C:/Users/Joginder Bairwa/Jupyter/CyberSec/CSRIC_Best_Practices.csv')
fcc_csv.head()

,BP Number,Priority,Description,Network Type(s),Industry Role(s),Keywords,Public Safety and Disaster,Reference,cable,internet/Data,satellite,wireless,wireline,Service Provider,Network Operator,"Priority (1,2,3)",Equipment Supplier,Property Manager,Government,Public Safety
0,12-10-0436,Highly Important,"Network Operators, Service Providers, and Publ...",Cable; Internet/Data; Satellite; Wireless; Wir...,Service Provider; Network Operator; Public Saf...,Network Operations; Procedures;,True,NaN,True,True,True,True,True,True,True,2.0,False,False,False,True
1,12-10-0437,Highly Important,Network Operators and Service Providers should...,Internet/Data;,Service Provider; Network Operator;,Cyber Security; Network Operations; Network Pr...,False,NaN,False,True,False,False,False,True,True,2.0,False,False,False,False
2,12-10-0440,Highly Important,Network Operators and Service Providers should...,Internet/Data;,Service Provider; Network Operator;,Industry Cooperation; Network Operations;,False,NaN,False,True,False,False,False,True,True,2.0,False,False,False,False
3,12-10-0447,Important,Network Operators and Service Providers should...,Cable; Internet/Data; Satellite; Wireless; Wir...,Service Provider; Network Operator;,Liaison; Network Operations;,False,Note: This Best practice could impact 9-1-1 op...,True,True,True,True,True,True,True,1.0,False,False,False,False
4,12-10-0448,Highly Important,"Equipment Suppliers should where feasible, pr...",Cable; Internet/Data; Satellite; Wireless; Wir...,Equipment Supplier;,Hardware; Network Elements; Network Provisioni...,False,NaN,True,True,True,True,True,False,False,2.0,True,False,False,False


In [8]:
# Here is a function for basic exploratory data analysis:
def eda(dataframe):
    # Replace any blank spaces w/ a underscore.
    dataframe.columns = dataframe.columns.str.replace(" ", "_")
    # Checks for the null values.
    print("missing values{}".format(dataframe.isnull().sum().sum()))
    # Checks the data frame range size.
    print("dataframe index: {}".format(dataframe.index))
    # Checks for data types of the columns within the data frame.
    print("dataframe types: {}".format(dataframe.dtypes))
    # Checks the shape of the data frame.
    print("dataframe shape: {}".format(dataframe.shape))
    # Gives us any statistical information of the data frame.
    print("dataframe describe: {}".format(dataframe.describe()))
    # Gives us the duplicated data of the data frame. 
print("duplicates{}".format(dataframe[dataframe.duplicated()].sum()))
    # A for loop that does this for every single column & their 
    # values within our data frame giving us all unique values.
    for item in dataframe:
        print(item)
        print(dataframe[item].nunique())
# Let's apply this function to our entire data frame.
eda(fcc_csv)


IndentationError: unexpected indent (<ipython-input-8-0b2b9cbb5ce6>, line 19)

In [9]:
# Here's a function to convert NaN's in the data set to 'None' for 
# string objects.
# Just pass in the entire data frame.
def convert_str_nan(data):
    return data.astype(object).replace(np.nan, 'None', inplace = True)
convert_str_nan(fcc_csv)

In [12]:
# Let's rename the 'Priority_(1,2,3)' column so we can utilize it.
fcc_csv.rename(columns = {
    'Priority_(1,2,3)': 'Priorities'
},
inplace = True)
# Let's view the values & how the correspond to the 'Priority' 
# column.
fcc_csv['Priorities'].value_counts()
# We notice that we have an unbalanced classification problem.
# Let's group the "Highly Important" (2) & "Critical" (3) aspects 
# because that's where we can make recommendations.
# Let's double check that it worked.
fcc_csv['Priorities'] = [0 if i == 1 else 1 for i in fcc_csv['Priorities']]
fcc_csv['Priorities'].value_counts()

KeyError: 'Priorities'